In [3]:
import os
from dotenv import load_dotenv
# load_dotenv() os.getenv("GOOGLE_API_KEY")
import google.generativeai as genai
import json
import pandas as pd
import traceback

e:\SOFTWARES\Anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-pro')


In [19]:
response=model.generate_content("Who is the fouder of google what was the founders first discovery how was google formed")

In [21]:
print(response.text)

**Founders of Google:**

* **Larry Page**
* **Sergey Brin**


**First Discovery:**

In 1996, while Page and Brin were PhD students at Stanford University, they developed a search engine called **BackRub**. This search engine was unique in that it used a "backlink" algorithm to rank web pages, rather than the traditional "keyword" algorithm. The backlink algorithm counted the number of other web pages that linked to a particular page, and then ranked the page higher if it had more backlinks. This algorithm was more effective at finding relevant and authoritative web pages than the traditional keyword algorithm, and it quickly gained popularity among Stanford students and faculty.


**Formation of Google:**

In 1998, Page and Brin incorporated Google Inc. The company's initial headquarters was a garage in Menlo Park, California. Google quickly grew in popularity, and by 2000, it was the most popular search engine in the world. In 2004, Google went public, and the company's stock price ha

In [24]:
# from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
# from langchain.callbacks import get_openai_callback
import PyPDF2

In [30]:
from langchain_google_genai import GoogleGenerativeAI
api_key=os.getenv("GOOGLE_API_KEY")

In [31]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key)
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

**Pros of Python**

* **Easy to learn:** Python is a relatively easy language to learn, making it a good choice for beginners. The syntax is simple and straightforward, and there are a lot of resources available to help you get started.
* **Versatile:** Python can be used for a wide variety of tasks, including web development, data science, and machine learning. This makes it a good choice for developers who want to work in a variety of fields.
* **High-level:** Python is a high-level language, which means that it is closer to human language than some other programming languages. This makes it easier to read and write code, and it can also help to reduce errors.
* **Interpreted:** Python is an interpreted language, which means that it does not need to be compiled before it can be run. This makes it a good choice for developing prototypes or scripts that you want to run quickly.
* **Large community:** Python has a large and active community of developers, which means that there are a lo

In [32]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [33]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [34]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [36]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [37]:

TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [38]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [39]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [40]:
#Sequential chain using output from quiz_chain as input for review _cahin
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [41]:
file_path=r"experiment\texts.txt"
file_path

'experiment\\texts.txt'

In [42]:
with open(file_path, 'r') as file:
    TEXT = file.read()

FileNotFoundError: [Errno 2] No such file or directory: 'experiment\\texts.txt'